# Deploying Iris-detection model using Vertex AI


### Install Vertex AI SDK for Python and other required packages



In [1]:
# Vertex SDK for Python
! pip3 install --upgrade --quiet  google-cloud-aiplatform

### Set Google Cloud Project Information

In [2]:
PROJECT_ID = "iitmbs-mlops"
LOCATION = "us-central1"

### Set GCS Information

In [3]:
BUCKET_URI = f"gs://iitmbs-mlops-21f1000344"

### Initialize Vertex AI SDK for Python

In [4]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Import the required libraries

In [5]:
import os
import sys

### Timestamp

In [13]:
# Create timestamp
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

## Simple Decision Tree model
Build a Decision Tree model on iris data

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pandas.plotting import parallel_coordinates
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics

### Load Data from GCS Bucket

In [15]:
from google.cloud import storage
import pandas as pd
import io

BUCKET_NAME = "iitmbs-mlops-21f1000344"
DATA_PATH = "iris_data/iris.csv"

# Initialize storage client
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob(DATA_PATH)

# Download data as string and read into pandas
data_bytes = blob.download_as_bytes()
iris_df = pd.read_csv(io.BytesIO(data_bytes))

iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [16]:
train, test = train_test_split(iris_df, test_size = 0.2, stratify = iris_df['species'], random_state = 42)
X_train = train[['sepal_length','sepal_width','petal_length','petal_width']]
y_train = train.species
X_test = test[['sepal_length','sepal_width','petal_length','petal_width']]
y_test = test.species

In [17]:
mod_dt = DecisionTreeClassifier(max_depth = 3, random_state = 1)
mod_dt.fit(X_train,y_train)
prediction=mod_dt.predict(X_test)
print('The accuracy of the Decision Tree is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test)))

The accuracy of the Decision Tree is 0.967


In [18]:
# Save model locally
import joblib

joblib.dump(mod_dt, f"artifacts/{timestamp}-model.joblib")

['artifacts/20250925-145240-model.joblib']

### Upload Model Artifacts to GCS

In [19]:
MODEL_ARTIFACT_DIR = f"my-models/{timestamp}-iris-classifier-week-1" # Path to model artifacts within the cloud storage bucket.

# Upload to GCS
blob = bucket.blob(f"{MODEL_ARTIFACT_DIR}/model.joblib")
blob.upload_from_filename(f"artifacts/{timestamp}-model.joblib")

print(f"Artifacts saved to gs://{BUCKET_NAME}/{MODEL_ARTIFACT_DIR}")

Artifacts saved to gs://iitmbs-mlops-21f1000344/my-models/20250925-145240-iris-classifier-week-1
